In [831]:
import pandas as pd
import numpy as np
import duckdb

## DuckDB

### Initializing a DuckDB database

In [832]:
con = duckdb.connect(database=':memory:')

In [833]:
con.execute('''
    CREATE TABLE training_set AS SELECT * FROM read_csv_auto('train-1.csv')
''')

In [834]:
con.execute("SELECT * FROM training_set").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True
...,...,...,...,...,...,...,...,...,...
958,9955,tt9558612,PM Náréndrá Mớdi,PM Narendra Modi,2019,\N,136,7005.0,False
959,9960,tt9598172,Sáving Léningrád,NaN,2019,\N,96,2200.0,False
960,9977,tt9691136,Shadow in the Cloud,NaN,2020,\N,83,22617.0,False
961,9979,tt9695258,So My Grandma's a Lesbian!,Salir del ropero,2019,\N,94,1054.0,False


In [835]:
con.execute('''
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-2.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-3.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-4.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-5.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-6.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-7.csv');
    INSERT INTO training_set SELECT * FROM read_csv_auto('train-8.csv')
''')

In [836]:
con.execute("SELECT * FROM training_set").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True
...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,NaN,2019,\N,87,12951.0,False
7955,9981,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False
7956,9982,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,\N,111,4144.0,True


### Checking the number of null values in the columns

In [837]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE primaryTitle IS NULL
''').fetchdf()

,count_star()
0,0


In [838]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE originalTitle IS NULL
''').fetchdf()

,count_star()
0,3988


In [839]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE startYear = '\\N'
''').fetchdf()

,count_star()
0,786


In [840]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE runtimeMinutes = '\\N'
''').fetchdf()

,count_star()
0,13


In [841]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE numVotes IS NULL
''').fetchdf()

,count_star()
0,790


## Data Cleaning

In [842]:
def execute(query):
    result = con.execute(query).fetchdf()
    return result

### 1) Replace missing values in startYear column

In [843]:
def replace_missing_startYear(input_name):
    query = '''
        UPDATE ''' + input_name + '''
        SET startYear = endYear
        WHERE startYear = '\\N'
    '''
    return query

In [844]:
execute(replace_missing_startYear('training_set'))

,Count
0,786


In [845]:
con.execute("SELECT * FROM training_set WHERE startYear = '\\N'").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label


### 2) Dropping endYear column

Now that the missing values in startYear have been replaced by the values of endYear, this latter column is no longer necessary. We can drop this column.

In [846]:
def drop_endYear(input_name):
    query = '''
        ALTER TABLE ''' + input_name + '''
        DROP COLUMN endYear
    '''
    return query

In [847]:
execute(drop_endYear('training_set'))

,Success


In [848]:
con.execute("SELECT * FROM training_set").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,Die Puppe,1919,66,1898.0,True
1,7,tt0011841,Way Down East,Way Down East,1920,145,5376.0,True
2,9,tt0012494,Déstiny,Der müde Tod,1921,97,5842.0,True
3,25,tt0015163,The Navigator,The Navigator,1924,59,9652.0,True
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,93,17887.0,True
...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,NaN,2019,87,12951.0,False
7955,9981,tt9741310,Slaxx,Slaxx,2020,77,2464.0,False
7956,9982,tt9742392,Kindred,Kindred,2020,101,1719.0,False
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,111,4144.0,True


### 3) Replace missing values in runtimeMinutes column

First, we can substitute the null values with 0 and set it to type integer so subsequent computations are easy to do. Then, we use the yearly average to impute the missing values - so we replace by the average of `runtimeMins` for movies belonging to the same year.

In [849]:
def convert_runtimeMins(input_name):
    query = '''
        UPDATE ''' + input_name + ''' SET runtimeMinutes = 0 WHERE runtimeMinutes = '\\N';
        ALTER TABLE ''' + input_name + ''' ALTER COLUMN runtimeMinutes SET DATA TYPE INTEGER;
    '''
    return query

In [850]:
def calculate_missing_runtimeMins(input_name):
    query = '''
        UPDATE ''' + input_name + ''' m1 
        SET runtimeMinutes = (
          SELECT AVG(runtimeMinutes) as yearly_mean 
          FROM ''' + input_name + ''' m2 
          WHERE m1.startYear = m2.startYear AND runtimeMinutes > 0 
          GROUP BY m2.startYear
        )
        WHERE runtimeMinutes = 0;
    '''
    return query

In [851]:
execute(convert_runtimeMins('training_set'))

,Success


In [852]:
execute(calculate_missing_runtimeMins('training_set'))

,Count
0,13


In [853]:
con.execute('''
    SELECT COUNT(*) FROM training_set WHERE runtimeMinutes = '\\N'
''').fetchdf()

,count_star()
0,0


In [854]:
con.execute("SELECT runtimeMinutes FROM training_set WHERE runtimeMinutes = 0").fetchdf()

,runtimeMinutes


### 4) Replace missing values in numVotes column

In [855]:
con.execute("SELECT * FROM training_set WHERE numVotes IS NULL").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,numVotes,label
0,118,tt0023973,Thé Éáglé ánd thé Háwk,NaN,1933,73,NaN,True
1,119,tt0023986,Émplớyéés' Éntráncé,NaN,1933,75,NaN,True
2,163,tt0027478,The Crime of Monsieur Lange,Le crime de Monsieur Lange,1936,80,NaN,True
3,180,tt0028333,Swing Timé,NaN,1936,103,NaN,True
4,466,tt0040626,My Dear Secretary,My Dear Secretary,1948,94,NaN,False
...,...,...,...,...,...,...,...,...
785,9446,tt7134096,The Rhythm Section,The Rhythm Section,2020,109,NaN,False
786,9664,tt8017136,Tớny: My Méntớr thé Sériál Killér,Tony,2018,124,NaN,True
787,9803,tt8671462,Invoking 5,Invoking 5,2018,90,NaN,False
788,9806,tt8694228,Mikhael,NaN,2019,150,NaN,False


#### 4a) Using trimmed mean

Similar to the previous one, we can substitute the null values in `numVotes` with 0 and set it to type integer. Then, we can use a trimmed average of the `numVotes` column, excluding the smallest and largest values so that the mean is less skewed.

In [ ]:
def convert_numVotes(input_name):
    query = '''
        UPDATE ''' + input_name + ''' SET numVotes = 0 WHERE numVotes IS NULL;
        ALTER TABLE ''' + input_name + ''' ALTER COLUMN numVotes SET DATA TYPE INTEGER;
    '''
    return query

In [ ]:
execute(convert_numVotes('training_set'))

In [421]:
def calculate_missing_numVotes(input_name):
    query = '''
        UPDATE ''' + input_name + ''' m1 
        SET numVotes = (
          SELECT (SUM(numVotes) - MIN(numVotes) - MAX(numVotes)) / CAST(COUNT(*)-2 as FLOAT) as trimmed_mean 
          FROM ''' + input_name + ''' m2 
          WHERE numVotes > 0
        )
        WHERE numVotes = 0;
    '''
    return query

In [423]:
execute(calculate_missing_numVotes('training_set'))

,Count
0,790


In [424]:
con.execute("SELECT numVotes FROM training_set WHERE numVotes IS NULL").fetchdf()

,numVotes


In [425]:
con.execute("SELECT numVotes FROM training_set WHERE numVotes = 0").fetchdf()

,numVotes


#### 4b) Using a supervised learning algorithm

We can use a supervised ML algorithm to impute the missing values in the numVotes column.

In [828]:
from sklearn.linear_model import LinearRegression

def predict_missing_numVotes(input_name):
    df_train = con.execute('''
        SELECT * FROM ''' + input_name + '''
        WHERE numVotes != 0
    ''').fetchdf()
    X_train = df_train[['startYear', 'runtimeMinutes']]
    y_train = df_train['numVotes']
    
    df_test = con.execute('''
        SELECT * FROM ''' + input_name + '''
        WHERE numVotes IS NULL
    ''').fetchdf()
    X_test = df_test[['startYear', 'runtimeMinutes']]
    
    # Train an SVM classifier
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Make predictions on the test set and evaluate the performance
    ## [1 2 2 0 0 5 0] + [3 3 3] --> [1 2 2 3 3 5 3]
    y_pred = model.predict(X_test)
    preds = pd.DataFrame(y_pred, columns = ['numVotes'])
    print(preds)
    
    df = con.execute('''
        SELECT * FROM ''' + input_name + '''
    ''').fetchdf()
    df['numVotes'] = df['numVotes'].mask(df['numVotes'].isna(), preds['numVotes'])
    
    con.execute('''
        DROP TABLE IF EXISTS ''' + input_name + '''
    ''')
    con.execute('''
        CREATE TABLE ''' + input_name + ''' AS SELECT * FROM df;
    ''')
#     con.execute('''
#         ALTER TABLE ''' + input_name + ''' ALTER COLUMN numVotes TYPE INTEGER USING CAST(numVotes AS INTEGER)
#     ''')

In [829]:
predict_missing_numVotes('training_set')

         numVotes
0    -2591.581747
1    -1065.554418
2     3085.054721
3    20634.369002
4    15109.409293
..            ...
785  34607.593880
786  45829.104967
787  19886.640378
788  65779.307180
789  34719.440819

[790 rows x 1 columns]


In [830]:
con.execute("SELECT * FROM training_set WHERE numVotes IS NULL").fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,runtimeMinutes,numVotes,label
0,8420,tt4065552,Tuntematon sotilas,Tuntematon sotilas,2017,180,NaN,True
1,8424,tt4074928,Realive,Realive,2016,112,NaN,False
2,8468,tt4164468,Warriors of the Rainbow: Seediq Bale II,Sai de ke · ba lai: Cai hong qiao,2011,132,NaN,True
3,9063,tt5746054,Spy Cát,NaN,2018,92,NaN,False
4,9218,tt6198246,The Great War,NaN,2019,108,NaN,False
...,...,...,...,...,...,...,...,...
707,9446,tt7134096,The Rhythm Section,The Rhythm Section,2020,109,NaN,False
708,9664,tt8017136,Tớny: My Méntớr thé Sériál Killér,Tony,2018,124,NaN,True
709,9803,tt8671462,Invoking 5,Invoking 5,2018,90,NaN,False
710,9806,tt8694228,Mikhael,NaN,2019,150,NaN,False


In [738]:
con.execute("SELECT numVotes FROM training_set WHERE numVotes = 0").fetchdf()

,numVotes


### 5) Dropping the originalTitle column

Since over 50% of the values in originalTitle are null and the primaryTitle already contains the main information regarding the movie name, we can discard this column from our dataset.

In [426]:
def drop_originalTitle(input_name):
    query = '''
        ALTER TABLE ''' + input_name + '''
        DROP COLUMN originalTitle
    '''
    return query

In [427]:
execute(drop_originalTitle('training_set'))

,Success


### 6) Checking duplicate rows

In [430]:
con.execute('''
    SELECT * FROM training_set 
    GROUP BY *
    HAVING COUNT(*) > 1
''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label


In [431]:
con.execute('''
    SELECT tconst, primaryTitle FROM training_set
    GROUP BY tconst, primaryTitle
    HAVING COUNT(*) > 1
''').fetchdf()

,tconst,primaryTitle


In [432]:
con.execute('''
    SELECT * FROM training_set 
    WHERE primaryTitle = 'Sabrina'
''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label
0,9650,tt7981492,Sabrina,2018,113,1662,False
1,620,tt0047437,Sabrina,1954,113,62736,True


In [433]:
con.execute('''
    SELECT * FROM training_set
    WHERE primaryTitle IN (
        SELECT primaryTitle FROM training_set
        GROUP BY primaryTitle
        HAVING COUNT(*) > 1
    )
''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label
0,38,tt0016220,The Phantom of the Opera,1925,93,17887,True
1,125,tt0024216,King Kong,1933,100,83177,True
2,239,tt0031647,Midnight,1939,94,4904,True
3,279,tt0033152,The Thief of Bagdad,1940,106,12840,True
4,400,tt0038355,The Big Sleep,1946,114,83357,True
...,...,...,...,...,...,...,...
213,8120,tt3462710,Unforgettable,2017,100,15087,False
214,8392,tt4008758,Black,2015,95,4152,True
215,9651,tt7984766,The King,2019,140,110160,True
216,9699,tt8144778,The Redeemed and the Dominant: Fittest on Earth,2018,119,1347,True


In [434]:
con.execute('''
    SELECT * FROM training_set
    WHERE primaryTitle = 'The Phantom of the Opera'
''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label
0,38,tt0016220,The Phantom of the Opera,1925,93,17887,True
1,3303,tt0119889,The Phantom of the Opera,1998,99,5390,False


In [435]:
con.execute("SELECT * FROM training_set").fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label
0,4,tt0010600,The Doll,1919,66,1898,True
1,7,tt0011841,Way Down East,1920,145,5376,True
2,9,tt0012494,Déstiny,1921,97,5842,True
3,25,tt0015163,The Navigator,1924,59,9652,True
4,38,tt0016220,The Phantom of the Opera,1925,93,17887,True
...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,2019,87,12951,False
7955,9981,tt9741310,Slaxx,2020,77,2464,False
7956,9982,tt9742392,Kindred,2020,101,1719,False
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,2020,111,4144,True


### Adding external data to this dataset

In [436]:
con.execute('''CREATE TABLE movielens_data AS SELECT * FROM read_csv_auto('movielens_data.csv')''')

In [437]:
con.execute('''SELECT * FROM movielens_data''').fetchdf()

,id,original_title,popularity,title,vote_average,release_year
0,862,Toy Story,21.946943,Toy Story,7.700000,1995
1,8844,Jumanji,17.015539,Jumanji,6.900000,1995
2,15602,Grumpier Old Men,11.712900,Grumpier Old Men,6.500000,1995
3,31357,Waiting to Exhale,3.859495,Waiting to Exhale,6.100000,1995
4,11862,Father of the Bride Part II,8.387519,Father of the Bride Part II,5.700000,1995
...,...,...,...,...,...,...
45284,439050,رگ خواب,0.072051,Subdue,4.000000,1900
45285,111109,Siglo ng Pagluluwal,0.178241,Century of Birthing,9.000000,2011
45286,67758,Betrayal,0.903007,Betrayal,3.800000,2003
45287,227506,Satana likuyushchiy,0.003503,Satan Triumphant,6.750000,1917


In [438]:
def add_external_columns(input_name):
    query = '''
        CREATE TABLE merged_''' + input_name + ''' AS 
        SELECT ''' + input_name + '''.*, movielens_data.popularity, movielens_data.vote_average FROM 
        ''' + input_name + ''' LEFT JOIN movielens_data 
        ON ''' + input_name + '''.primaryTitle = movielens_data.title
        AND ''' + input_name + '''.startYear = movielens_data.release_year;
    '''
    return query

In [439]:
execute(add_external_columns('training_set'))

,Count
0,7959


In [440]:
con.execute('''SELECT * FROM merged_training_set''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average
0,2,tt0009369,Mickey,1918,93,1119,False,0.211008,7.5
1,4,tt0010600,The Doll,1919,66,1898,True,NaN,NaN
2,5,tt0011439,The Mark of Zorro,1920,79,2439,True,0.336801,6.3
3,6,tt0011607,The Parson's Widow,1920,94,1264,True,NaN,NaN
4,7,tt0011841,Way Down East,1920,145,5376,True,1.649987,6.2
...,...,...,...,...,...,...,...,...,...
7954,9995,tt9850344,Night Shift,2020,98,1325,False,NaN,NaN
7955,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,2020,111,4144,True,NaN,NaN
7956,9997,tt9900782,Kaithi,2019,145,29179,True,NaN,NaN
7957,9998,tt9904802,Enemy Lines,2020,92,29179,False,NaN,NaN


### Impute missing values in popularity column

Using same technique as before, use a trimmed average of popularity column as a whole, excluding smallest & largest values.

In [443]:
con.execute('''
    SELECT COUNT(*) FROM merged_training_set WHERE popularity IS NULL
''').fetchdf()

,count_star()
0,4011


In [444]:
def convert_popularity(input_name):
    query = '''
        UPDATE ''' + input_name + ''' SET popularity = 0 WHERE popularity IS NULL;
        ALTER TABLE ''' + input_name + ''' ALTER COLUMN popularity SET DATA TYPE FLOAT;
    '''
    return query

In [445]:
def calculate_missing_popularity(input_name):
    query = '''
        UPDATE ''' + input_name + ''' m1 
        SET popularity = (
          SELECT (SUM(popularity) - MIN(popularity) - MAX(popularity)) / CAST(COUNT(*)-2 as FLOAT) as trimmed_mean 
          FROM ''' + input_name + ''' m2 
          WHERE popularity > 0
        )
        WHERE popularity = 0;
    '''
    return query

In [446]:
execute(convert_popularity('merged_training_set'))

,Success


In [447]:
execute(calculate_missing_popularity('merged_training_set'))

,Count
0,4011


In [448]:
con.execute('''SELECT * FROM merged_training_set WHERE popularity IS NULL''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average


In [449]:
con.execute('''SELECT * FROM merged_training_set WHERE popularity = 0''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average


In [450]:
con.execute('''SELECT * FROM merged_training_set''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average
0,2,tt0009369,Mickey,1918,93,1119,False,0.211008,7.5
1,4,tt0010600,The Doll,1919,66,1898,True,4.837576,NaN
2,5,tt0011439,The Mark of Zorro,1920,79,2439,True,0.336801,6.3
3,6,tt0011607,The Parson's Widow,1920,94,1264,True,4.837576,NaN
4,7,tt0011841,Way Down East,1920,145,5376,True,1.649987,6.2
...,...,...,...,...,...,...,...,...,...
7954,9995,tt9850344,Night Shift,2020,98,1325,False,4.837576,NaN
7955,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,2020,111,4144,True,4.837576,NaN
7956,9997,tt9900782,Kaithi,2019,145,29179,True,4.837576,NaN
7957,9998,tt9904802,Enemy Lines,2020,92,29179,False,4.837576,NaN


### Impute missing values in vote_average column

In [451]:
con.execute('''
    SELECT COUNT(*) FROM merged_training_set WHERE vote_average IS NULL
''').fetchdf()

,count_star()
0,4011


In [452]:
def convert_vote_average(input_name):
    query = '''
        UPDATE ''' + input_name + ''' SET vote_average = 0 WHERE vote_average IS NULL;
        ALTER TABLE ''' + input_name + ''' ALTER COLUMN vote_average SET DATA TYPE FLOAT;
    '''
    return query

In [453]:
execute(convert_vote_average('merged_training_set'))

,Success


In [454]:
def calculate_missing_vote_average(input_name):
    query = '''
        UPDATE ''' + input_name + ''' 
        SET vote_average = (
          SELECT MEDIAN(vote_average) as median 
          FROM ''' + input_name + ''' 
          WHERE vote_average > 0
        )
        WHERE vote_average = 0;
    '''
    return query

In [455]:
execute(calculate_missing_vote_average('merged_training_set'))

,Count
0,4011


In [456]:
con.execute('''SELECT * FROM merged_training_set WHERE vote_average IS NULL''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average


In [457]:
con.execute('''SELECT * FROM merged_training_set WHERE vote_average = 0''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average


In [458]:
con.execute('''SELECT * FROM merged_training_set''').fetchdf()

,column0,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average
0,2,tt0009369,Mickey,1918,93,1119,False,0.211008,7.5
1,4,tt0010600,The Doll,1919,66,1898,True,4.837576,6.3
2,5,tt0011439,The Mark of Zorro,1920,79,2439,True,0.336801,6.3
3,6,tt0011607,The Parson's Widow,1920,94,1264,True,4.837576,6.3
4,7,tt0011841,Way Down East,1920,145,5376,True,1.649987,6.2
...,...,...,...,...,...,...,...,...,...
7954,9995,tt9850344,Night Shift,2020,98,1325,False,4.837576,6.3
7955,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,2020,111,4144,True,4.837576,6.3
7956,9997,tt9900782,Kaithi,2019,145,29179,True,4.837576,6.3
7957,9998,tt9904802,Enemy Lines,2020,92,29179,False,4.837576,6.3


## Model training

In [459]:
df_1 = con.execute('''SELECT * FROM training_set''').fetchdf()
df_1 = df_1.set_index('column0')
df_1

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label
column0,,,,,,
4,tt0010600,The Doll,1919,66,1898,True
7,tt0011841,Way Down East,1920,145,5376,True
9,tt0012494,Déstiny,1921,97,5842,True
25,tt0015163,The Navigator,1924,59,9652,True
38,tt0016220,The Phantom of the Opera,1925,93,17887,True
...,...,...,...,...,...,...
9966,tt9625664,Trauma Center,2019,87,12951,False
9981,tt9741310,Slaxx,2020,77,2464,False
9982,tt9742392,Kindred,2020,101,1719,False


In [461]:
df_2 = con.execute('''SELECT * FROM merged_training_set''').fetchdf()
df_2 = df_2.set_index('column0')
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average
column0,,,,,,,,
2,tt0009369,Mickey,1918,93,1119,False,0.211008,7.5
4,tt0010600,The Doll,1919,66,1898,True,4.837576,6.3
5,tt0011439,The Mark of Zorro,1920,79,2439,True,0.336801,6.3
6,tt0011607,The Parson's Widow,1920,94,1264,True,4.837576,6.3
7,tt0011841,Way Down East,1920,145,5376,True,1.649987,6.2
...,...,...,...,...,...,...,...,...
9995,tt9850344,Night Shift,2020,98,1325,False,4.837576,6.3
9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,2020,111,4144,True,4.837576,6.3
9997,tt9900782,Kaithi,2019,145,29179,True,4.837576,6.3


In [462]:
df_2 = df_2.reindex(df_1.index)
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,label,popularity,vote_average
column0,,,,,,,,
4,tt0010600,The Doll,1919,66,1898,True,4.837576,6.3
7,tt0011841,Way Down East,1920,145,5376,True,1.649987,6.2
9,tt0012494,Déstiny,1921,97,5842,True,4.837576,6.3
25,tt0015163,The Navigator,1924,59,9652,True,3.031803,7.3
38,tt0016220,The Phantom of the Opera,1925,93,17887,True,11.083684,7.0
...,...,...,...,...,...,...,...,...
9966,tt9625664,Trauma Center,2019,87,12951,False,4.837576,6.3
9981,tt9741310,Slaxx,2020,77,2464,False,4.837576,6.3
9982,tt9742392,Kindred,2020,101,1719,False,4.837576,6.3


In [463]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split the data into training and test sets
X = df_2[['startYear', 'runtimeMinutes', 'numVotes', 'popularity', 'vote_average']]
y = df_2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an SVM classifier
clf = SVC()
clf.fit(X_train, y_train)

# Make predictions on the test set and evaluate the performance
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print(f'Test set accuracy: {accuracy:.2f}')

Test set accuracy: 0.56


In [464]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score

# Split data into features and labels
X = df_2[['startYear', 'runtimeMinutes', 'numVotes', 'popularity', 'vote_average']]
y = df_2['label']

# Train-test split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# Train logistic regression model
lr_model = LogisticRegression(max_iter=1000)
lr_model.fit(train_X, train_y)
lr_pred = lr_model.predict(test_X)
lr_acc = accuracy_score(test_y, lr_pred)
print("Logistic Regression Accuracy: {:.4f}".format(lr_acc))

# Train random forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(train_X, train_y)
rf_pred = rf_model.predict(test_X)
rf_acc = accuracy_score(test_y, rf_pred)
print("Random Forest Accuracy: {:.4f}".format(rf_acc))

# Train gradient boosted trees model
gbt_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gbt_model.fit(train_X, train_y)
gbt_pred = gbt_model.predict(test_X)
gbt_acc = accuracy_score(test_y, gbt_pred)
print("Gradient Boosted Trees Accuracy: {:.4f}".format(gbt_acc))

Logistic Regression Accuracy: 0.8097
Random Forest Accuracy: 0.8046
Gradient Boosted Trees Accuracy: 0.8241


## Validation set

In [465]:
con.execute('''
    CREATE TABLE validation_set AS SELECT * FROM read_csv_auto('validation_hidden.csv')
''')

In [466]:
con.execute('''SELECT * FROM validation_set''').fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes
0,0,tt0003740,Cabiria,NaN,1914,\N,148,3452.0
1,1,tt0008663,A Man There Was,Terje Vigen,1917,\N,65,1882.0
2,3,tt0010307,J'accuse!,NaN,1919,\N,166,1692.0
3,18,tt0014429,Safety Last!,Safety Last!,1923,\N,74,19898.0
4,27,tt0015175,Die Nibelungen: Siegfried,NaN,1924,\N,143,5676.0
...,...,...,...,...,...,...,...,...
950,9974,tt9686154,You Will Die at 20,NaN,2019,\N,103,2106.0
951,9976,tt9690328,Pápér Spidérs,Paper Spiders,2020,\N,109,NaN
952,9980,tt9735790,Me You Madness,Me You Madness,2021,\N,98,1056.0
953,9984,tt9769668,Tughlaq Durbar,Tughlaq Durbar,2021,\N,145,1430.0


In [468]:
## Replace missing startYear values and drop endYear
execute(replace_missing_startYear('validation_set'))
execute(drop_endYear('validation_set'))

## Replace missing runtimeMins values
execute(convert_runtimeMins('validation_set'))
execute(calculate_missing_runtimeMins('validation_set'))

## Replace missing numVotes values
execute(convert_numVotes('validation_set'))
execute(calculate_missing_numVotes('validation_set'))

## Dropping the originalTitle column
execute(drop_originalTitle('validation_set'))

execute(add_external_columns('validation_set'))

execute(convert_popularity('merged_validation_set'))
execute(calculate_missing_popularity('merged_validation_set'))
execute(convert_vote_average('merged_validation_set'))
execute(calculate_missing_vote_average('merged_validation_set'))

,Count
0,444


In [469]:
con.execute('''
    SELECT COUNT(*) FROM validation_set WHERE primaryTitle IS NULL
''').fetchdf()

,count_star()
0,0


In [470]:
con.execute('''
    SELECT COUNT(*) FROM validation_set WHERE startYear = '\\N'
''').fetchdf()

,count_star()
0,0


In [471]:
con.execute('''
    SELECT COUNT(*) FROM validation_set WHERE runtimeMinutes = '\\N'
''').fetchdf()

,count_star()
0,0


In [472]:
con.execute('''
    SELECT COUNT(*) FROM validation_set WHERE numVotes IS NULL
''').fetchdf()

,count_star()
0,0


### Predictions on validation set

In [475]:
df_1 = con.execute('''SELECT * FROM validation_set''').fetchdf()
df_1 = df_1.set_index('column0')
df_1

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes
column0,,,,,
0,tt0003740,Cabiria,1914,148,3452
1,tt0008663,A Man There Was,1917,65,1882
3,tt0010307,J'accuse!,1919,166,1692
18,tt0014429,Safety Last!,1923,74,19898
27,tt0015175,Die Nibelungen: Siegfried,1924,143,5676
...,...,...,...,...,...
9974,tt9686154,You Will Die at 20,2019,103,2106
9976,tt9690328,Pápér Spidérs,2020,109,25068
9980,tt9735790,Me You Madness,2021,98,1056


In [476]:
df_2 = con.execute('''SELECT * FROM merged_validation_set''').fetchdf()
df_2 = df_2.set_index('column0')
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,popularity,vote_average
column0,,,,,,,
0,tt0003740,Cabiria,1914,148,3452,0.704028,6.8
1,tt0008663,A Man There Was,1917,65,1882,1.322761,6.6
3,tt0010307,J'accuse!,1919,166,1692,0.834679,7.9
18,tt0014429,Safety Last!,1923,74,19898,4.394823,7.7
27,tt0015175,Die Nibelungen: Siegfried,1924,143,5676,1.827403,7.7
...,...,...,...,...,...,...,...
9974,tt9686154,You Will Die at 20,2019,103,2106,4.691750,6.2
9976,tt9690328,Pápér Spidérs,2020,109,25068,4.691750,6.2
9980,tt9735790,Me You Madness,2021,98,1056,4.691750,6.2


In [477]:
df_2 = df_2.reindex(df_1.index)
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,popularity,vote_average
column0,,,,,,,
0,tt0003740,Cabiria,1914,148,3452,0.704028,6.8
1,tt0008663,A Man There Was,1917,65,1882,1.322761,6.6
3,tt0010307,J'accuse!,1919,166,1692,0.834679,7.9
18,tt0014429,Safety Last!,1923,74,19898,4.394823,7.7
27,tt0015175,Die Nibelungen: Siegfried,1924,143,5676,1.827403,7.7
...,...,...,...,...,...,...,...
9974,tt9686154,You Will Die at 20,2019,103,2106,4.691750,6.2
9976,tt9690328,Pápér Spidérs,2020,109,25068,4.691750,6.2
9980,tt9735790,Me You Madness,2021,98,1056,4.691750,6.2


In [478]:
X_val = df_2[['startYear', 'runtimeMinutes', 'numVotes', 'popularity', 'vote_average']]

# SVM
y_val_preds = clf.predict(X_val)

# Logistic regression model
lr_val_preds = lr_model.predict(X_val)

# Random forest model
rf_val_preds = rf_model.predict(X_val)

# Gradient boosted trees model
gbt_val_preds = gbt_model.predict(X_val)

In [479]:
val_preds = [str(pred) for pred in gbt_val_preds]

In [480]:
pd.DataFrame(val_preds).to_csv('val_predictions.csv', index=False, header=False)

## Test set

In [481]:
con.execute('''
    CREATE TABLE test_set AS SELECT * FROM read_csv_auto('test_hidden.csv')
''')

In [482]:
con.execute('''SELECT * FROM test_set''').fetchdf()

,column0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes
0,22,tt0014972,He Who Gets Slapped,He Who Gets Slapped,1924,\N,95,3654.0
1,23,tt0015016,The Iron Horse,NaN,1924,\N,150,2136.0
2,26,tt0015174,Die Nibelungen: Kriemhild's Revenge,NaN,1924,\N,129,4341.0
3,28,tt0015214,At 3:25,NaN,\N,1925,59,1724.0
4,34,tt0015863,Go West,NaN,1925,\N,69,4188.0
...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,One Piece: Stampede,NaN,2019,\N,101,5109.0
1082,9943,tt9441638,The Big Ugly,NaN,2020,\N,106,5780.0
1083,9948,tt9495690,Págálpánti,Pagalpanti,2019,\N,149,2331.0
1084,9950,tt9519642,The Wedding Unplanner,NaN,2020,\N,110,NaN


In [483]:
## Replace missing startYear values and drop endYear
execute(replace_missing_startYear('test_set'))
execute(drop_endYear('test_set'))

## Replace missing runtimeMins values
execute(convert_runtimeMins('test_set'))
execute(calculate_missing_runtimeMins('test_set'))

## Replace missing numVotes values
execute(convert_numVotes('test_set'))
execute(calculate_missing_numVotes('test_set'))

## Dropping the originalTitle column
execute(drop_originalTitle('test_set'))

execute(add_external_columns('test_set'))

execute(convert_popularity('merged_test_set'))
execute(calculate_missing_popularity('merged_test_set'))
execute(convert_vote_average('merged_test_set'))
execute(calculate_missing_vote_average('merged_test_set'))

,Count
0,534


In [484]:
con.execute('''
    SELECT COUNT(*) FROM test_set WHERE primaryTitle IS NULL
''').fetchdf()

,count_star()
0,0


In [485]:
con.execute('''
    SELECT COUNT(*) FROM test_set WHERE startYear = '\\N'
''').fetchdf()

,count_star()
0,0


In [486]:
con.execute('''
    SELECT COUNT(*) FROM test_set WHERE runtimeMinutes = '\\N'
''').fetchdf()

,count_star()
0,0


In [487]:
con.execute('''
    SELECT COUNT(*) FROM test_set WHERE numVotes IS NULL
''').fetchdf()

,count_star()
0,0


### Predictions on test set

In [488]:
df_1 = con.execute('''SELECT * FROM test_set''').fetchdf()
df_1 = df_1.set_index('column0')
df_1

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes
column0,,,,,
22,tt0014972,He Who Gets Slapped,1924,95,3654
23,tt0015016,The Iron Horse,1924,150,2136
26,tt0015174,Die Nibelungen: Kriemhild's Revenge,1924,129,4341
28,tt0015214,At 3:25,1925,59,1724
34,tt0015863,Go West,1925,69,4188
...,...,...,...,...,...
9942,tt9430698,One Piece: Stampede,2019,101,5109
9943,tt9441638,The Big Ugly,2020,106,5780
9948,tt9495690,Págálpánti,2019,149,2331


In [489]:
df_2 = con.execute('''SELECT * FROM merged_test_set''').fetchdf()
df_2 = df_2.set_index('column0')
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,popularity,vote_average
column0,,,,,,,
22,tt0014972,He Who Gets Slapped,1924,95,3654,0.719924,7.2
23,tt0015016,The Iron Horse,1924,150,2136,0.990037,7.1
26,tt0015174,Die Nibelungen: Kriemhild's Revenge,1924,129,4341,0.828949,7.5
28,tt0015214,At 3:25,1925,59,1724,4.663778,6.4
34,tt0015863,Go West,1925,69,4188,1.014248,6.8
...,...,...,...,...,...,...,...
9942,tt9430698,One Piece: Stampede,2019,101,5109,4.663778,6.4
9943,tt9441638,The Big Ugly,2020,106,5780,4.663778,6.4
9948,tt9495690,Págálpánti,2019,149,2331,4.663778,6.4


In [490]:
df_2 = df_2.reindex(df_1.index)
df_2

,tconst,primaryTitle,startYear,runtimeMinutes,numVotes,popularity,vote_average
column0,,,,,,,
22,tt0014972,He Who Gets Slapped,1924,95,3654,0.719924,7.2
23,tt0015016,The Iron Horse,1924,150,2136,0.990037,7.1
26,tt0015174,Die Nibelungen: Kriemhild's Revenge,1924,129,4341,0.828949,7.5
28,tt0015214,At 3:25,1925,59,1724,4.663778,6.4
34,tt0015863,Go West,1925,69,4188,1.014248,6.8
...,...,...,...,...,...,...,...
9942,tt9430698,One Piece: Stampede,2019,101,5109,4.663778,6.4
9943,tt9441638,The Big Ugly,2020,106,5780,4.663778,6.4
9948,tt9495690,Págálpánti,2019,149,2331,4.663778,6.4


In [491]:
X_test = df_2[['startYear', 'runtimeMinutes', 'numVotes', 'popularity', 'vote_average']]

# SVM
y_test_preds = clf.predict(X_test)

# Logistic regression model
lr_test_preds = lr_model.predict(X_test)

# Random forest model
rf_test_preds = rf_model.predict(X_test)

# Gradient boosted trees model
gbt_test_preds = gbt_model.predict(X_test)

In [492]:
test_preds = [str(pred) for pred in gbt_test_preds]

In [493]:
pd.DataFrame(gbt_test_preds).to_csv('test_predictions.csv', index=False, header=False)